# Datasets, Pandas DataFrames and Databases

## Abstract

Para la adquisición de datos, en general se dispone de archivos que contienen los datos almacenados. Sin embargo, el modo más común para persistir información es en bases de datos.
En esta notebook se realizarán operaciones básicas con bases de datos. En particular se utiliza SQLite interactuando con DataFrames Pandas.

## 1. Introduccion

### Acerca de SQLite

"<i>SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world  with more than a trillon of databeses (1e12) in use.</i>" (del Sitio oficial: https://www.sqlite.org/index.html).<br>
SQLite permite crear una base de datos SQL en tiempo de ejecución con solo importar una librería y crear el archivo. Mediante sentencias SQL puede, insertar datos, consultarlos, modificar tablas, etc. Puede navegar las bases de datos con https://sqlitebrowser.org/. Y puede eliminar la DB símplemente eliminando el archivo.
<br><br>
Para descargar e instalar Sqlite Browser: https://sqlitebrowser.org/dl/ <br>
<br>
Para instalar el módulo SQLlite:<br>
- conda install sqlite3 (instalar desde Anaconda)<br>
- pip install sqlite3 (instalar desde Terminal - Linux / MacOS - o desde DOS / Powershell - Windows)<br>



### Librerias

In [6]:
import sqlite3
import pandas as pd
import datetime as datetime
import os
#
# El separador presentado abajo es multi OS.
separador = os.path.sep

En este ejemplo, cargaremos un dataset desde un archivo, crearemos una base de datos y almacenaremos el daframe en la base de datos.

## 2. DDL

Dentro de SQL, un sublenguaje es DDL (Data Definition Languaje). Consiste en un conjunto de sentencias para la definición y modificación de la base de datos y sus tablas.

In [7]:
# Cargamos los datos desde un CSV
archivoStrokesCsv='datasets'+str(separador)+'train_strokes.csv'
df=pd.read_csv(archivoStrokesCsv)

In [18]:
# Crearemos una base de datos, el cursor y la conexión
nombreDB='datasets'+str(separador)+'train_strokes.db'
conexion = sqlite3.connect(nombreDB, uri=True)
cursor = conexion.cursor()

### Crear Tablas

In [19]:
# Esta tabla se crea para ser eliminada (y poder mostrarlo)
sentencia="""CREATE TABLE employees(
            id integer PRIMARY KEY, 
            name text, 
            salary real, 
            department text, 
            position text, 
            startDate text
            )"""
try:
    cursor.execute(sentencia)
    print("se creo la tabla employees")
except sqlite3.OperationalError:
    print("Tabla existente")

# Esta tabla se crea para persistir.
sentence="""CREATE TABLE strokes(
            id integer PRIMARY KEY AUTOINCREMENT,
            gender text,
            age real,
            hypertension real,
            heart_disease real,
            ever_married text,
            work_type text,
            Residence_type text,
            avg_glucose_level real,
            bmi real,
            smoking_status text,
            stroke real
            )"""
try:
    cursor.execute(sentence)
    print("se creo la tabla strokes")
except sqlite3.OperationalError:
    print("Tabla existente")

se creo la tabla employees
se creo la tabla strokes


### Eliminar Tablas

In [20]:
# Eliminación de tabla creada
sentencia="""drop table employees"""
try:
    cursor.execute(sentencia)
    print("se elimino la tabla employees")
except:
    print("Error operacional")

se elimino la tabla employees


#### Confirmación: Navegamos con DB Browser

Abrimos la DB "train_strokes.db" y abrimos la tabla "strokes"
<img src="files/browseDB_01.png" alt="SQLite3 DB Browser">

## 3. DML

DML (Data Manipulation Languaje) es un sub-lenguaje de SQL. Es un conjunto de sentencias para la manipulación de los datos almacenados.

In [21]:
# Reemplazar NAN por ceros en todo el Df
df.fillna(0)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,0,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,0,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43395,56196,Female,10.0,0,0,No,children,Urban,58.64,20.4,never smoked,0
43396,5450,Female,56.0,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0
43397,28375,Female,82.0,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0
43398,27973,Male,40.0,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0


### Insert: Insertar los registros del Df en la Tabla de la DB

In [22]:
inicio=datetime.datetime.now()
print("La carga del Df en la DB puede demorar unos 30s aprox. dependiendo de su PC")
# Vaciamos la tabla para que no aparezcan errores de ids repetidos
cursor.execute("DELETE FROM strokes")
conexion.commit()
for i in range (len(df)):
    if(i/10000==int(i/10000)):
        print("Insertados ",i," registros de ",len(df))
    sentence = """INSERT OR IGNORE INTO strokes VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"""
    var00=i+1
    var01=df.iloc[i,1]
    var02=int(df.iloc[i,2])
    var03=int(df.iloc[i,3])
    var04=int(df.iloc[i,4])
    var05=df.iloc[i,5]
    var06=df.iloc[i,6]
    var07=df.iloc[i,7]
    var08=int(df.iloc[i,8])
    var09=float(df.iloc[i,9])
    var10=df.iloc[i,10]
    var11=int(df.iloc[i,11])
    cursor.execute(sentence,(var00,var01,var02,var03,var04,var05,var06,var07,var08,var09,var10,var11))
conexion.commit()
fin=datetime.datetime.now()
print("Demora: ",fin-inicio)

La carga del Df en la DB puede demorar unos 30s aprox. dependiendo de su PC
Insertados  0  registros de  43400
Insertados  10000  registros de  43400
Insertados  20000  registros de  43400
Insertados  30000  registros de  43400
Insertados  40000  registros de  43400
Demora:  0:00:28.017479


### Select

In [23]:
# SELECT
sentence="SELECT * FROM strokes"
cursor.execute(sentence)

In [24]:
# Presentar en pantalla la cantidad de registros
registros = cursor.fetchall()
df2=pd.DataFrame(registros)
print("Cant. de registros elegidos: ",len(df2))

Cant. de registros elegidos:  43400


In [25]:
# Presentar en pantalla a los registros iniciales
print(df2.head())

   0       1     2    3    4    5             6      7      8     9   \
0   1    Male   3.0  0.0  0.0   No      children  Rural   95.0  18.0   
1   2    Male  58.0  1.0  0.0  Yes       Private  Urban   87.0  39.2   
2   3  Female   8.0  0.0  0.0   No       Private  Urban  110.0  17.6   
3   4  Female  70.0  0.0  0.0  Yes       Private  Rural   69.0  35.9   
4   5    Male  14.0  0.0  0.0   No  Never_worked  Rural  161.0  19.1   

                10   11  
0             None  0.0  
1     never smoked  0.0  
2             None  0.0  
3  formerly smoked  0.0  
4             None  0.0  


### Cierre de Conexion

In [26]:
# CIERRE DE CONEXION
cursor.close()
conexion.close()